<a href="https://colab.research.google.com/github/DDD26/tum-adlr-ws20-02/blob/main/quadratic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## basic setup ##
!pip install stable-baselines3[extra]
!apt-get install ffmpeg freeglut3-dev xvfb

     |████████████████████████████████| 153kB 5.6MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
freeglut3-dev set to manually installed.
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 783 kB of archives.
After this operation, 2,266 kB of additional disk space will be used.
Err:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.7
  404  Not Found [IP: 91.189.88.142 80]
E: Failed to fetch http://archive.ubuntu.com/ubuntu/pool/universe/x/xorg-server/xvfb_1.19.6-1ubuntu4.7_amd64.deb  404  Not Found [IP: 91.189.88.142 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [ ]:
## import ##
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import scipy.ndimage as ndimage
from scipy.signal import convolve2d
import torch

In [ ]:
import numpy as np
import gym
from gym import spaces

class OPEnv(gym.Env):
  def __init__(self, size=10,dim = 2):
    super(OPEnv, self).__init__()
    self.size = size
    self.dim = dim
    
    self.pos = np.full((self.dim,),size-1).astype(np.float32)
    self.action_space = spaces.Box(low=-self.size, high=self.size,
                                    shape=(dim,), dtype=np.float32)
    self.observation_space = spaces.Box(low=-4*self.size, high=4*self.size,
                                        shape=(dim,), dtype=np.float32)

  def reset(self):
    self.pos = np.full((self.dim,),self.size-1).astype(np.float32)
    return 2*self.pos

  def step(self, action):
    self.pos += action
    self.pos = np.clip(self.pos, -self.size, self.size)
    
    done = bool(np.all(self.pos == 0))
    reward = np.float(-np.sum(self.pos*self.pos))
    info = {}

    return 2*self.pos, reward, done, info

  def close(self):
    pass

In [ ]:
from stable_baselines3 import PPO, A2C # DQN coming soon
from stable_baselines3.common.cmd_util import make_vec_env
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO,SAC
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

env = OPEnv(dim=10)
#env = make_vec_env(lambda: env, n_envs=1)
check_env(env, warn=True)
model = PPO('MlpPolicy', env, gamma=1, verbose=1).learn(10000)

/usr/local/lib/python3.6/dist-packages/stable_baselines3/common/env_checker.py:232: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


Using cpu device
Wrapping the env in a DummyVecEnv.
-----------------------------------------
| time/                   |             |
|    fps                  | 1330        |
|    iterations           | 1           |
|    time_elapsed         | 1           |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.017240554 |
|    clip_fraction        | 0.112       |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.23       |
|    explained_variance   | -1.19e+03   |
|    learning_rate        | 0.0003      |
|    loss                 | 6.08e+05    |
|    n_updates            | 30          |
|    policy_gradient_loss | -0.0198     |
|    std                  | 0.989       |
|    value_loss           | 8.73e+05    |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 882         |
|    iterations         

In [ ]:
#obs = env.reset()

env.pos = 10*np.random.rand(10) ## choose another start point to see weather the policy generalize
obs = env.pos*2                              ## choose another start point to see weather the policy generalize

n_steps = 200
for step in range(n_steps):
  action,_ = model.predict(obs, deterministic=True)
  obs, reward, done, info = env.step(action)
  print(env.pos)
  if done:
    print("Goal reached!", "reward=", reward,"step",step)
    break


[ 3.81840638  7.20551586  3.26647049  6.90280304  9.63400151  5.58746682
  8.22811808  8.55992511  3.1015512  -0.01417878]
[ 3.77111653  7.12554237  3.17062027  6.81835624  9.5302007   5.49500949
  8.14061955  8.52154097  3.0312204  -0.04825139]
[ 3.72393349  7.04521979  3.07584019  6.7334514   9.42613398  5.40339952
  8.05224052  8.48269345  2.96113212 -0.08200169]
[ 3.67685512  6.96453394  2.98212487  6.64807827  9.32181523  5.31265946
  7.96300394  8.44337198  2.89131759 -0.11541765]
[ 3.62988342  6.88347115  2.88946631  6.56222775  9.21725715  5.22280851
  7.87293384  8.40356411  2.82180824 -0.14848741]
[ 3.5830238   6.80201842  2.79785374  6.47589177  9.11247109  5.13386237
  7.78205505  8.3632559   2.75263531 -0.1811997 ]
[ 3.53628424  6.72016332  2.70727353  6.38906303  9.00746695  5.04583307
  7.69039308  8.32243259  2.68382953 -0.21354407]
[ 3.4896744   6.63789411  2.61770936  6.30173492  8.90225315  4.95872909
  7.59797383  8.28107906  2.61542079 -0.24551112]
[ 3.44320464  6.